# Import Calibrated Images into GEE as a Cloud GeoTiff Backed ImageCollection
https://developers.google.com/earth-engine/Earth_Engine_asset_from_cloud_geotiff

## Set Up Environment

In [ ]:
import ee
from google.auth.transport.requests import AuthorizedSession
ee.Initialize()
session = AuthorizedSession(ee.data.get_persistent_credentials())

In [ ]:
import re
from google.cloud import storage
from pprint import pprint

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Import Images into GEE
Note: an empty ImageCollection with the right name needs to be created ahead of time

In [ ]:
# Get paths to calibrated images
dir_path = ("planet_processing/data/nitze_regions/mad_output/calibrated_files/")

gcs_files = storage_client.list_blobs("abrupt_thaw", prefix = dir_path)
image_paths = []
for file in gcs_files:
    if re.match('.*SR.*tif$', file.name):
        image_paths.append('gs://abrupt_thaw/' + file.name)

ids = [path.split('/')[-1][0:-4] for path in image_paths]
print(len(image_paths))
print(image_paths[0])
print(ids[0])

In [ ]:
# Earth Engine enabled Cloud Project.
project_folder = 'abruptthawmapping'

for image, current_id in zip(image_paths, ids):
    # The request
    request = {
      'type': 'IMAGE',
      'gcs_location': {
        'uris': image
      }
    }

    # A folder (or ImageCollection) name and the new asset name.
    asset_id = 'nitze_regions_imagery_calibrated/{}'
    asset_id = asset_id.format(current_id)

    url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'

    response = session.post(
      url = url.format(project_folder, asset_id),
      data = json.dumps(request)
    )

    pprint(json.loads(response.content))
